In [2]:
from __Detection.detection_api import DetectionAPI
from __Tracking.tracking_api import TrackerAPI
from homo_graphy import PlanProjector
import cv2

In [3]:
class Args:
    track_thresh = 0.5
    match_thresh = 0.5
    track_buffer = 60
    mot20 = False
    cpu_workers = 16   # 듀얼 CPU 적극 활용
    chunk_sec   = 10.0 # 15~30초 권장
args = Args()

In [4]:
detector = DetectionAPI()
tracker = TrackerAPI(args=args, detector=detector)

Loads checkpoint by local backend from path: /workspace/PretrainedModel_by_JeonYT/vehicle/epoch_54.pth
Loads checkpoint by local backend from path: /workspace/PretrainedModel_by_JeonYT/worker/epoch_100.pth


In [5]:
# 1) 도면 + (선택) H 로드
plan_path = "/workspace/assets/25082_homograph_coordinate-plane.jpg"
plan_pts  = [
            (3588, 412),    #point1
            (3588, 1036),   #point2
            (3588, 1657),   #point3
            (2225, 412),    #point4
            (2225, 1036),   #point5
            (2225, 1657),   #point6
            (861, 412),     #point7
            (861, 1036),    #point8
            (861, 1657),    #point9
             ]

### Cam#1

In [ ]:
cam1_tracklets = tracker.trackingVideo(
    video_path="/workspace/datasets/homography_experiment/1030-1100/2025-08-27_10_30_00_cam#1.mp4",
    save_path="/workspace/results/tracking_result1.mp4",
    trail_len=100
)

# 2) 대응점으로 H 추정 (리스트 안에 '튜플' 형태) --> 예시 나중에 바꿔야함
projector1 = PlanProjector(cam1_tracklets, trail_len=60, trail_ttl=30, line_thickness=4, point_radius=6)
cam1_pts = [
    (1390, 521),  #point1
    (1618, 552),  #point2
    (1784, 578),  #point3
    (1112, 564),  #point4
    (1434,620),   #point5
    (1709,668),   #point6
    (481,651),    #point7
    (852,809),    #point8
    (1393,1007)   #point9
    ]
H, mask = projector1.fit_homography(cam1_pts, plan_pts, ransac_thresh=3.0)

# 4-1) 예시, 한 프레임만 투영(10번 프레임) & 저장
projected, canvas = projector1.projection(cam1_tracklets[10], mode="bottom-center", draw=True)
cv2.imwrite("/workspace/results/plan_proj_frame1.jpg", canvas)
# 4-2) 전체 프레임 미니맵 비디오 저장
projector1.save_video(cam1_tracklets, "/workspace/results/plan_projection1.mp4", fps=30, mode="bottom-center")

/opt/conda/envs/mmyolo/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


KeyboardInterrupt: 

### Cam#2

In [ ]:
cam2_tracklets = tracker.trackingVideo(
    video_path="/workspace/datasets/homography_experiment/1030-1100/2025-08-27_10_30_00_cam#2.mp4",
    save_path="/workspace/results/tracking_result2.mp4",
    trail_len=100
)

# 2) 대응점으로 H 추정 (리스트 안에 '튜플' 형태) --> 예시 나중에 바꿔야함
projector2 = PlanProjector(cam2_tracklets, trail_len=60, trail_ttl=30, line_thickness=4, point_radius=6)
cam3_pts = [
    (1020,1027), #point1
    (476,898),   #point2
    (294,826),   #point3
    (1348,790),  #point4
    (1017,772),  #point5
    (813,753),   #point6
    (1437,716),  #point7
    (1213,713),  #point8
    (1058,707)   #point9
    ]
H, mask = projector2.fit_homography(cam3_pts, plan_pts, ransac_thresh=3.0)

# 4-1) 예시, 한 프레임만 투영(10번 프레임) & 저장
projected, canvas = projector2.projection(cam2_tracklets[10], mode="bottom-center", draw=True)
cv2.imwrite("/workspace/results/plan_proj_frame2.jpg", canvas)
# 4-2) 전체 프레임 미니맵 비디오 저장
projector2.save_video(cam2_tracklets, "/workspace/results/plan_projection2.mp4", fps=30, mode="bottom-center")


 ### Cam#3

In [ ]:
cam3_tracklets = tracker.trackingVideo(
    video_path="/workspace/mounted_datasets/Hyundai_dataset/aug1/c1/video.mp4",
    save_path="/workspace/results/tracking_result3.mp4",
    trail_len=100
)

# 2) 대응점으로 H 추정 (리스트 안에 '튜플' 형태) --> 예시 나중에 바꿔야함
projector3 = PlanProjector(cam3_tracklets, trail_len=60, trail_ttl=30, line_thickness=4, point_radius=6)
cam3_pts = [
    (140,309),   #point1
    (291,315),   #point2
    (398,320),   #point3
    (213,376),   #point4
    (440,377),   #point5
    (588,378),   #point6
    (467,596),   #point7
    (826,521),   #point8
    (965,484)    #point9
]
H, mask = projector3.fit_homography(cam3_pts, plan_pts, ransac_thresh=3.0)

# 4-1) 예시, 한 프레임만 투영(10번 프레임) & 저장
projected, canvas = projector3.projection(cam3_tracklets[10], mode="bottom-center", draw=True)
cv2.imwrite("/workspace/results/plan_proj_frame3.jpg", canvas)
# 4-2) 전체 프레임 미니맵 비디오 저장
projector3.save_video(cam3_tracklets, "/workspace/results/plan_projection3.mp4", fps=30, mode="bottom-center")
